<a href="https://colab.research.google.com/github/permarisk/earth_obs_methods_2025/blob/main/Resample_precipitation_catchment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rioxarray

In [ ]:
import rioxarray as rxr
import xarray as xr
import rasterio
from rasterio.enums import Resampling
import dask
from rasterio.enums import Resampling
import gdown

In [ ]:
#'GOOGLE_DRIVE_LINK' to NetCDF file
url = 'https://drive.google.com/file/d/1Z9-YDydbsxFBq4Do43KaB3-Lyzm9dkpd/view?usp=drive_link'
# Download the NetCDF file
file_path = 'precipitation_grid.nc'
gdown.download(url, file_path, quiet=False, fuzzy=True)


#'GOOGLE_DRIVE_LINK' to NetCDF file
url = 'https://drive.google.com/file/d/1bpJmPDlsAIEqI6XQ9x90dUa1kYzU--ZJ/view?usp=drive_link'
# Download the Tiff file
file_path = 'catchment_grid.tif'
gdown.download(url, file_path, quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1Z9-YDydbsxFBq4Do43KaB3-Lyzm9dkpd
From (redirected): https://drive.google.com/uc?id=1Z9-YDydbsxFBq4Do43KaB3-Lyzm9dkpd&confirm=t&uuid=3811967b-d090-421d-b0f6-00601f14db93
To: /content/precipitation_grid.nc
100%|██████████| 325M/325M [00:02<00:00, 147MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bpJmPDlsAIEqI6XQ9x90dUa1kYzU--ZJ
To: /content/catchment_grid.tif
100%|██████████| 3.15M/3.15M [00:00<00:00, 38.2MB/s]


'catchment_grid.tif'

In [ ]:
# File paths
precip_nc_file = "precipitation_grid.nc"  # NetCDF precipitation data
catchment_tif_file = "catchment_grid.tif"  # Higher-resolution raster
output_tif_file = "precipitation_resampled.tif"  # Output NetCDF

# Load the precipitation dataset (NetCDF)
precip_xr = xr.open_dataset(precip_nc_file)
# Load the catchment raster (GeoTIFF)
catchment = rxr.open_rasterio(catchment_tif_file)

In [ ]:
# Ensure both datasets have the same CRS (EPSG:4326)
precip = precip_xr['tp']
# Rename 'lon' and 'lat' to 'x' and 'y' (required for rioxarray)
precip = precip.rename({"lon": "x", "lat": "y"})
# Explicitly set spatial dimensions
precip = precip.rio.set_spatial_dims(x_dim="x", y_dim="y") # Replace "longitude" and "latitude" with the actual dimension names if different.
precip = precip.rio.write_crs("EPSG:4326")
catchment = catchment.rio.write_crs("EPSG:4326")

In [ ]:
# Clip precipitation data to match the spatial extent of the catchment
precip_resampled = precip.rio.clip_box(*catchment.rio.bounds())
precip_resampled

<xarray.DataArray 'tp' (time: 5114, y: 15, x: 24)> Size: 15MB
[1841040 values with dtype=float64]
Coordinates:
  * time         (time) datetime64[ns] 41kB 2003-01-01 2003-01-02 ... 2016-12-31
  * y            (y) float64 120B 28.28 28.27 28.26 28.25 ... 28.18 28.17 28.16
  * x            (x) float64 192B -16.8 -16.8 -16.79 ... -16.63 -16.62 -16.61
    spatial_ref  int64 8B 0

In [ ]:
# Reproject and resample precipitation to match the catchment resolution
#precip_resampled = precip_resampled.rio.reproject_match(catchment, resampling=Resampling.nearest)

# Use reproject_match with Dask for memory efficiency
with dask.config.set(scheduler="threads"):  # Allows parallel processing
    precip_resampled = precip.rio.reproject_match(catchment, resampling=Resampling.nearest)


In [ ]:
# Convert back to NetCDF and save the resampled dataset
precip_resampled.to_raster(output_tif_file)

# Print dataset details
print("Original Precipitation Grid Shape:", precip[precip_var].shape)
print("Catchment Grid Shape:", catchment.shape)
print("Resampled Precipitation Grid Shape:", precip_resampled.shape)
print(f"Resampled precipitation data saved to: {output_nc_file}")